In [0]:
spark.conf.set("fs.azure.account.key.riyazsparkpractice24.dfs.core.windows.net",
               "f/qu7WlrAu/vakDff9iixYzUyAVj+MmxkDdA5xHDDqa2yi2hY52z63t8lkBfy/j7MuZZ4+Ox5/Cj+AStmiJsyw==")




In [0]:

display(dbutils.fs.ls("abfss://tamingbigdatawithpyspark@riyazsparkpractice24.dfs.core.windows.net/ml-100k/u.data"))

In [0]:
import collections
df = spark.read.text("abfss://tamingbigdatawithpyspark@riyazsparkpractice24.dfs.core.windows.net/ml-100k/u.data")

rdd = df.rdd
lines = rdd.map(lambda row: row['value'])
ratings = lines.map(lambda x: x.split()[2])

result = ratings.countByValue()
sortedResults = collections.OrderedDict(sorted(result.items()))
print(type(sortedResults))
for key, value in sortedResults.items():
    print("%s %i" % (key, value))


In [0]:
df = spark.read.text("abfss://tamingbigdatawithpyspark@riyazsparkpractice24.dfs.core.windows.net/fakefriends.csv")

rdd = df.rdd

lines = rdd.map(lambda row: row['value'])

def parseline(x):
    age = int(x.split(',')[2])
    number_of_friends = int(x.split(',')[3])
    return(age,number_of_friends)

lines_splitted = lines.map(parseline)

average_friends_by_age = lines_splitted.mapValues(lambda x:(x,1)).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .mapValues(lambda x: x[0]/x[1])

average_friends_by_age  = average_friends_by_age.sortByKey()

for i in average_friends_by_age.collect():
    print(i)


In [0]:
df = spark.read.text("abfss://tamingbigdatawithpyspark@riyazsparkpractice24.dfs.core.windows.net/1800.csv")

rdd = df.rdd

lines = rdd.map(lambda row: row['value'])

def parseline(x):
    stationid = x.split(',')[0]
    entrytype = x.split(',')[2]
    temprature = float(x.split(',')[3])*0.1*(9.0/5.0)+32.0
    return(stationid,entrytype,temprature)

parsedlines = lines.map(parseline)

filtered_rdd = parsedlines.filter(lambda x: x[1]=='TMIN')

stationtemps = filtered_rdd.map(lambda x :(x[0],x[2]))

mintemps = stationtemps.reduceByKey(lambda x,y:min(x,y))

for i in mintemps.collect():
    print(i[0],"\t{:.2f}F".format(i[1]))


In [0]:
df = spark.read.text("abfss://tamingbigdatawithpyspark@riyazsparkpractice24.dfs.core.windows.net/1800.csv")

rdd = df.rdd

lines = rdd.map(lambda row: row['value'])

def parseline(x):
    stationid = x.split(',')[0]
    entrytype = x.split(',')[2]
    temprature = float(x.split(',')[3])*0.1*(9.0/5.0)+32.0
    return(stationid,entrytype,temprature)

parsedlines = lines.map(parseline)

filtered_rdd = parsedlines.filter(lambda x: x[1]=='TMAX')

stationtemps = filtered_rdd.map(lambda x :(x[0],x[2]))

mintemps = stationtemps.reduceByKey(lambda x,y:max(x,y))

for i in mintemps.collect():
    print(i[0],"\t{:.2f}F".format(i[1]))


In [0]:
df = spark.read.text("abfss://tamingbigdatawithpyspark@riyazsparkpractice24.dfs.core.windows.net/Book")

rdd = df.rdd

lines = rdd.map(lambda row: row['value'])

words = lines.flatMap(lambda x:x.split())
wordcount = words.countByValue()

for i,j in wordcount.items(a):
    print(i,j)

In [0]:
import re 
df = spark.read.text("abfss://tamingbigdatawithpyspark@riyazsparkpractice24.dfs.core.windows.net/Book")

rdd = df.rdd

lines = rdd.map(lambda row: row['value'])


def normalizewords(x):
    return re.compile(r'\W+',re.UNICODE).split(x.lower())

words = lines.flatMap(normalizewords)
wordcount = words.countByValue()

for i in wordcount.items():
    print(i)

In [0]:
import re 
df = spark.read.text("abfss://tamingbigdatawithpyspark@riyazsparkpractice24.dfs.core.windows.net/Book")

rdd = df.rdd

lines = rdd.map(lambda row: row['value'])

def normalizewords(x):
    return re.compile(r'\W+',re.UNICODE).split(x.lower())

words = lines.flatMap(normalizewords)

wordcount = words.map(lambda x : (x,1)).reduceByKey(lambda x,y:x+y).map(lambda x:(x[1],x[0])).sortByKey(False).map(lambda x:(x[1],x[0]))

for i in wordcount.collect():
    print(i)




In [0]:
import re 
df = spark.read.text("abfss://tamingbigdatawithpyspark@riyazsparkpractice24.dfs.core.windows.net/customer-orders.csv")

rdd = df.rdd

lines = rdd.map(lambda row: row['value'])

def parseline(a):
    customer_id = int(a.split(',')[0])
    amount_spent = float(a.split(',')[2])
    return(customer_id,amount_spent)

customer_spent = lines.map(parseline)
total_spend_by_customer = customer_spent.reduceByKey(lambda x,y:x+y).map(lambda x:(x[1],x[0])).sortByKey(False).map(lambda x:(x[1],x[0]))

for i in total_spend_by_customer.collect():
    print(i[0],"\t{:.2f}".format(i[1]))
